In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
from PIL import Image, ImageDraw
import tensorflow as tf
import io
import glob
from tqdm import tqdm
import numpy as np
import logging
import argparse
import os
import json
import csv
import skimage.filters as filters

C:\Users\vinay\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vinay\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vinay\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vinay\Anaconda3\envs\PythonGPU\lib\site-packages\ten

In [2]:
def get_labels(fname="xView_train.geojson"):
    
    with open(fname) as f:
        data = json.load(f)
    
    coords = np.zeros((len(data['features']),4))
    chips = np.zeros((len(data['features'])),dtype="object")
    classes = np.zeros((len(data['features'])))
    
    for i in range(len(data['features'])):
        if data['features'][i]['properties']['bounds_imcoords'] != []:
            b_id = data['features'][i]['properties']['image_id']
            val = np.array([int(num) for num in data['features'][i]['properties']['bounds_imcoords'].split(",")])
            chips[i] = b_id
            classes[i] = data['features'][i]['properties']['type_id']
            coords[i] = val
        else:
            chips[i] = 'None'
            
    return coords, chips, classes

In [3]:
coords, chips, classes= get_labels()

In [4]:
coords

array([[2712., 1145., 2746., 1177.],
       [2720., 2233., 2760., 2288.],
       [2687., 1338., 2740., 1399.],
       ...,
       [2106.,  361., 2124.,  379.],
       [1917.,   38., 1958.,   64.],
       [2323.,   55., 2349.,   74.]])

In [5]:
chips

array(['2355.tif', '2355.tif', '2355.tif', ..., '389.tif', '389.tif',
       '389.tif'], dtype=object)

In [6]:
classes

array([73., 73., 73., ..., 73., 73., 73.])

In [7]:
def chip_image(img,coords,classes,shape=(500,500)):
    """
    Chip an image and get relative coordinates and classes.  Bounding boxes that pass into
        multiple chips are clipped: each portion that is in a chip is labeled. For example,
        half a building will be labeled if it is cut off in a chip. If there are no boxes,
        the boxes array will be [[0,0,0,0]] and classes [0].
        Note: This chip_image method is only tested on xView data-- there are some image manipulations that can mess up different images.
    Args:
        img: the image to be chipped in array format
        coords: an (N,4) array of bounding box coordinates for that image
        classes: an (N,1) array of classes for each bounding box
        shape: an (W,H) tuple indicating width and height of chips
    Output:
        An image array of shape (M,W,H,C), where M is the number of chips,
        W and H are the dimensions of the image, and C is the number of color
        channels.  Also returns boxes and classes dictionaries for each corresponding chip.
    """
    height,width,_ = img.shape
    wn,hn = shape
    
    w_num,h_num = (int(width/wn),int(height/hn))
    images = np.zeros((w_num*h_num,hn,wn,3))
    total_boxes = {}
    total_classes = {}
    
    k = 0
    for i in range(w_num):
        for j in range(h_num):
            x = np.logical_or( np.logical_and((coords[:,0]<((i+1)*wn)),(coords[:,0]>(i*wn))),
                               np.logical_and((coords[:,2]<((i+1)*wn)),(coords[:,2]>(i*wn))))
            out = coords[x]
            y = np.logical_or( np.logical_and((out[:,1]<((j+1)*hn)),(out[:,1]>(j*hn))),
                               np.logical_and((out[:,3]<((j+1)*hn)),(out[:,3]>(j*hn))))
            outn = out[y]
            out = np.transpose(np.vstack((np.clip(outn[:,0]-(wn*i),0,wn),
                                          np.clip(outn[:,1]-(hn*j),0,hn),
                                          np.clip(outn[:,2]-(wn*i),0,wn),
                                          np.clip(outn[:,3]-(hn*j),0,hn))))
            box_classes = classes[x][y]
        
            if out.shape[0] != 0:
                total_boxes[k] = out
                total_classes[k] = box_classes
            else:
                total_boxes[k] = np.array([[0,0,0,0]])
                total_classes[k] = np.array([0])
            
            chip = img[hn*j:hn*(j+1),wn*i:wn*(i+1),:3]
            images[k]=chip
            
            k = k + 1
    
    return images.astype(np.uint8),total_boxes,total_classes

In [8]:
def get_image(fname):    
    """
    Get an image from a filepath in ndarray format
    """
    return np.array(Image.open(fname))

In [9]:
import glob
image_folder= 'imagedata/'
fnames = glob.glob(image_folder + "*.tif")
fnames.sort()

In [10]:
new_chips=[]
new_classes=[]
new_coords=[]

In [11]:
for fname in fnames:
            
            name = fname.split("\\")[-1]
            arr = get_image(fname)
           # im,box,classes_final = chip_image(arr,coords[chips==name],classes[chips==name])
        #    for idx, image in enumerate(im):

                
#                 if not box[idx].any():
#                     continue
                
         #   Image.fromarray(image).save('process/img_%s_%s.png'%(name,idx))
            new_chips.extend([name]*len(coords[chips==name]))
            new_classes.extend(classes[chips==name])
            new_coords.extend(coords[chips==name])

                    
            

In [12]:
new_coords=np.array(new_coords)
new_classes= np.array(new_classes)
new_chips= np.array(new_chips)


In [13]:
new_coords=new_coords.T

In [14]:
train = pd.DataFrame({'image_names': new_chips, 'object_type': new_classes, 'xmin':new_coords[0].astype(int),\
                      'xmax':new_coords[2].astype(int),\
                        'ymin':new_coords[1].astype(int),'ymax':new_coords[3].astype(int)})
train

,image_names,object_type,xmin,xmax,ymin,ymax
0,1085.tif,73.0,-37,29,433,472
1,1085.tif,73.0,-21,41,507,536
2,1085.tif,73.0,-24,10,617,644
3,1085.tif,73.0,-49,17,652,682
4,1085.tif,73.0,-67,1,963,1036
...,...,...,...,...,...,...
5113,86.tif,19.0,478,493,2445,2458
5114,86.tif,11.0,762,956,1774,1915
5115,86.tif,19.0,100,141,1712,1730
5116,86.tif,64.0,1072,1088,2547,2568


In [15]:
data = pd.DataFrame()
data['format'] = train['image_names']


# data.reset_index(drop=True, inplace=True)
data

,format
0,1085.tif
1,1085.tif
2,1085.tif
3,1085.tif
4,1085.tif
...,...
5113,86.tif
5114,86.tif
5115,86.tif
5116,86.tif


In [16]:

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(data.shape[0]):
    data['format'][i] = str(data['format'][i]) + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + str(train['object_type'][i])

#data.to_csv('annotate.txt', header=None, index=None, sep=' ')

In [17]:
data.to_csv('annotate.txt', header=None, index=None, sep=' ')